# Visualizing Milky Way Dwarf Companions in DigitalSky Dark Matter

We will start with a catalog of 53 nearby galaxies provided by Alex Drlica-Wagner and Brian Nord of the Dark Energy Survey. In the visualization we will highlight those galaxies that were discovered by the Dark Energy Survey. We will then generate the files needed to visualize this catalog in the Digital Sky Dark Matter software that runs on the Iziko Planetarium and Digital Dome.

In [2]:
# Astropy imports
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import Distance
from astropy.table import Table, Column
from astropy.io import ascii
# Here we read the catalog which is in a .yaml file
import yaml
f = open('data/dwarfs.yaml')
# use safe_load instead load
dataMap = yaml.safe_load(f)
f.close()

In [22]:
#collapse the dictionary tree into something that can be converted into a simple table 
keys = ['name','ra', 'dec', 'distance','major_axis','refs']
flatDict=[]
for key in dataMap:
    keyDict = {x:dataMap[key][x] for x in keys}
    flatDict.append(keyDict)

dwarfTable=Table(flatDict)
dwarfTable

dec,distance,major_axis,name,ra,refs
float64,float64,float64,str24,float64,str25
-60.85,48.0,11.8,Tucana IV,0.73,\citep{1508.03622}
-50.28,125.9,1.2,Pictoris I,70.95,\citep{1503.02584}
-28.03,182.0,1.9,Columba I,82.86,\citep{1508.03622}
12.30639,254.0,3.4,Leo I,152.11708,\citep{Mateo:2007xh}
20.17528,35.0,3.4,Segue 2,34.81667,\citep{Kirby:2013isa}
-54.11,87.1,2.4,Horologium I,43.87,\citep{}
-1.61472,86.0,27.8,Sextans,153.2625,\citep{Walker:2008ax}
-31.99,134.0,1.7,Hydra II,185.43,\citep{1503.06216}
-27.6667,7.0,nan,Canis Major,108.14583,--


In [23]:
# Here we will add an additional column which is 1 if that galaxy was discovered by DES, 0 otherwise. 
# This column will be used to color the galaxies in the final visualizaton
import numpy as np
byDES=np.zeros(len(dwarfTable))
for i in range(len(dwarfTable['refs'])):
  if ('1508.03622' in dwarfTable['refs'][i]):
    byDES[i]=1.
  elif ('1503.02584' in dwarfTable['refs'][i]):
    byDES[i]=1.
dwarfTable.add_column(Column(byDES,name='byDES'))  
dwarfTable.add_column(Column(dwarfTable['ra']/15.,name='ra_h'))  
dwarfTable.remove_columns(['refs','ra'])  
dwarfTable

dec,distance,major_axis,name,byDES,ra_h
float64,float64,float64,str24,float64,float64
-60.85,48.0,11.8,Tucana IV,1.0,0.0486666666667
-50.28,125.9,1.2,Pictoris I,1.0,4.73
-28.03,182.0,1.9,Columba I,1.0,5.524
12.30639,254.0,3.4,Leo I,0.0,10.1411386667
20.17528,35.0,3.4,Segue 2,0.0,2.32111133333
-54.11,87.1,2.4,Horologium I,0.0,2.92466666667
-1.61472,86.0,27.8,Sextans,0.0,10.2175
-31.99,134.0,1.7,Hydra II,0.0,12.362
-27.6667,7.0,nan,Canis Major,0.0,7.209722


In [35]:
dwarfTable.show_in_browser(jsviewer=True)

### Write as VOTable
Digital Sky Dark Matter reads VOTables. Dark Matter can parse the UCDs, however, here we won't bother to set them. We will provide Dark Matter with a seperate xml file that tells it how to interpret the VOTable (which columns are coordinates, which are labels, which should be used to size or color the particles). We can now provide spherical coordinates (ra,dec,distance), no longer having to convert to cartesian.

In [25]:
# Write the VO Table
dwarfTable.write('data/MWdwarfs.xml',format='votable')

In [26]:
# Write the xml file that contains column descriptions
import xml.etree.cElementTree as ET

root = ET.Element("ParticleTable")
doc = ET.SubElement(root, "Columns")
ET.SubElement(doc, "AddField", data="ra_h", type="RA")
ET.SubElement(doc, "AddField", data="dec", type="DEC")
ET.SubElement(doc, "AddField", data="name", type="LABEL")
ET.SubElement(doc, "AddField", data="distance", type="DIST")
ET.SubElement(doc, "AddField", data="major_axis")
ET.SubElement(doc, "AddField", data="byDES")
tree = ET.ElementTree(root)
tree.write("data/MWdwarfs_fields.xml")

## Write a colormap file
Digital Sky Dark Matter reads colormaps in the partiview .cmap file. These can either be interpreted as continous or indexed colors. Here we will generate one with just two colors one for galaxies discovered by DES one for all others. 

In [25]:
cmapFile=open('data/DES.cmap', 'w')
cmapFile.write('2\n1 1 1 1\n1 0 0 1\n')
cmapFile.close()

# Tidal disruption of the Saggitarius Dwarf Galaxy
Here we will create the files necessary to visualize an N-body simulation of the tidal disruption of the Sagattarius Dwarf galaxy as described in Law and Majewski 2012 http://iopscience.iop.org/0004-637X/714/1/229/article Data files were taken from the website: http://www.astro.virginia.edu/~srm4n/Sgr/data.html
<img src="https://raw.githubusercontent.com/marksubbarao/pyWWT_AAS225/master/images/SgrStream.png">

In [12]:
#Import data set, keeping only the columns we need.
Stars = Table.read('data/SgrTriax_DYN.dat', format='ascii.basic')
Stars.keep_columns(["ra","dec","dist","Pcol"])
Stars.add_column(Column(Stars['ra']/15.,name='ra_h'))
Stars.remove_column('ra')
Stars

dec,dist,Pcol,ra_h
float64,float64,int32,float64
-42.1607,65.9751,8,19.7312
-31.0457,28.7055,-1,18.9714
-31.3875,24.1148,0,20.0002666667
-35.746,62.4169,8,18.6288666667
-38.2792,49.9106,7,17.8385333333
1.10254,47.1228,4,11.7928666667
33.1628,53.4371,4,6.96293333333
-0.34414,25.8991,5,4.77308666667
39.4735,44.6561,8,13.4354666667


In [14]:
# Write as VO Table
Stars.write('data/SGdwarf.xml',format='votable')

In [15]:
# Write the xml file that contains column descriptions
import xml.etree.cElementTree as ET

root = ET.Element("ParticleTable")
doc = ET.SubElement(root, "Columns")
ET.SubElement(doc, "AddField", data="ra_h", type="RA")
ET.SubElement(doc, "AddField", data="dec", type="DEC")
ET.SubElement(doc, "AddField", data="dist", type="DIST")
ET.SubElement(doc, "AddField", data="Pcol")
tree = ET.ElementTree(root)
tree.write("data/SGdwarf_fields.xml")

### Constructing th Colormap
Pcol indicates the pass when stars were stripped from the dwarf galaxy. We will constructing an identical colormap to the one used in the LM12 paper. Black is changed to grey for visability.

In [28]:
cmapFile=open('data/SG.cmap', 'w')
cmapFile.write('''8
0.5 0.5 0.5 1.0
1.0 0.5 0.0 1.0
1.0 0.5 0.0 1.0
1.0 0.0 1.0 1.0
1.0 0.0 1.0 1.0
0.0 1.0 1.0 1.0
0.0 1.0 1.0 1.0
0.0 1.0 0.1 1.0
''')
cmapFile.close()

In [3]:
#Import data set, keeping only the columns we need.
SgrStars = Table.read('data/SgrTriax_DYN.dat', format='ascii.basic')


In [4]:
SgrStars

lambda,beta,ra,dec,l,b,xgc,ygc,zgc,xsun,ysun,zsun,x4,y4,z4,u,v,w,dist,vgsr,mul,mub,mua,mud,Pcol,Lmflag
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,int32
11.2581,11.8798,295.968,-42.1607,357.131,-27.1087,-50.6537,-2.9399,-30.0635,63.3197,12.6044,13.5816,49.854,27.7916,12.6844,-14.2853,24.8315,133.139,65.9751,-49.0752,-0.660274,0.38155,-0.52177,-0.556147,8,-1
0.797464,1.93505,284.571,-31.0457,5.3522,-14.9475,-19.6132,2.58698,-7.40414,28.6863,0.399293,0.969288,19.2415,7.51085,0.072138,-224.773,-54.9545,192.7,28.7055,161.564,-2.24593,1.67382,-2.41743,-1.41488,-1,-1
13.9574,0.959365,300.004,-31.3875,9.78189,-27.4802,-13.0829,3.63479,-11.1276,23.3996,5.81569,0.403762,12.7937,11.4748,-0.493388,-283.358,-41.6733,169.269,24.1148,163.343,-2.76507,2.16152,-2.85939,-2.03511,0,-1
357.456,7.35559,279.433,-35.746,359.147,-12.8979,-52.8353,-0.905288,-13.9323,61.8422,-2.74746,7.99104,52.164,12.5428,7.09389,54.6785,24.1362,134.608,62.4169,-83.6895,-0.705156,0.374437,-0.631458,-0.488579,8,-1
348.843,12.2429,267.578,-38.2792,352.479,-5.6437,-41.2414,-6.50144,-4.90831,47.8537,-9.43825,10.5839,40.2289,2.64337,9.68675,131.053,56.2076,147.246,49.9106,-151.098,-0.813527,0.541365,-0.878448,-0.428056,7,-1
253.81,16.6704,176.893,1.10254,269.789,59.7997,8.08724,-23.7038,40.7269,-12.5868,-43.352,13.5179,-10.1192,-44.9829,12.6208,130.168,17.4374,-78.3508,47.1228,-76.2468,-0.619036,-1.02361,-1.03894,-0.592945,4,1
181.045,-4.04045,104.444,33.1628,183.285,15.6665,59.3674,-2.94803,14.43,-53.2954,-0.971759,-3.76521,-59.932,-13.8068,-4.66236,93.707,14.522,-58.5634,53.4371,73.4621,0.833309,-0.37154,-0.0245885,-0.912053,4,-1
140.82,18.4868,71.5963,-0.34414,197.791,-27.6848,29.8373,-7.00745,-12.0329,-19.0401,15.5177,8.21224,-30.7307,10.5368,7.31509,76.4606,-3.16514,-136.52,25.8991,128.754,1.61004,-0.454989,0.433123,-1.61606,5,2
255.591,-27.7557,201.532,39.4735,94.8154,75.7016,8.92582,10.9898,43.2728,-9.83366,-38.275,-20.7964,-8.68689,-39.3879,-21.6936,203.56,-34.1824,-127.967,44.6561,-128.195,1.10536,0.974351,-1.45737,-0.217377,8,-1
